In [2]:
#Import relevant libraries
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import requests
from time import sleep
from random import randint
import pandas as pd

### Note that the below function is now finalized and should work on any search
#### Note to self: we should pull a title of what the search is?

In [3]:
#Writing a single function to extract all title urls from an IMDB search
def url_extractor(search_url):
    #setting up initial BeautifulSoup object from websearch 
    init_resp = urllib.request.urlopen(search_url)
    init_soup = BeautifulSoup(init_resp, 'html.parser')
    #extract the number of films that the query returned. Used to confirm at end and generate each url 
    number_of_films = int(str(init_soup.find_all('div', class_='desc')[0].find_all('span')[0]).split(' ')[2].replace(',',''))
    print(number_of_films)
    #each page has 50 movies, so setting up a list to iterate through the pages, set up blank list to store final urls
    iterative_urls = [i for i in range(1,number_of_films, 50)]
    url_list = []
    #loop through the 50-spaced interger values to generate entire list of needed search urls 
    for i in iterative_urls:
        # set url
        url = search_url + '&start=' + str(i)
        #set up the BeautifulSoup object for this specific page of the search
        resp = urllib.request.urlopen(url)
        soup = BeautifulSoup(resp, 'html.parser')
        #generating list of all links on this page
        links = [a.get('href') for a in soup.find_all('a', href=True)]
        # printing out where we are in the query to monitor efficiency
        print('Running query from {} to {}'.format(i, i+49))
        # checking each link in each search page for title/tt keyword
        for link in links:
            if 'title/tt' in link:
                #when the length is 4 of the split title url, that means it is part of query 
                #when the length is 3, it means that the movie is ancillary to the actual search
                if len(link.split('/')) == 4:
                # format the resulting url in the correct manner and appending it to final list 
                    title_link = 'https://www.imdb.com' + '/' + link.split('/')[1] + '/' + link.split('/')[2]
                    if not title_link in url_list:
                        url_list.append(title_link)
                    else: 
                        continue 
                else: 
                    continue
            else:
                continue
    # Final test to make sure that the length of query equals the length of returned list and returning the final list
    if len(url_list) == number_of_films:
        print('All urls have been extracted successfully')
    else: 
        print('WARNING: The number of films in this query was {}, but {} urls were returned'.format(number_of_films, len(url_list)))
    return(url_list)

### Running a test dataset with the 480 length url above. Note that the number of urls and the length of the query are off by 1. 

In [4]:
urls = url_extractor('https://www.imdb.com/search/title/?title_type=feature&release_date=2019-01-01,2019-01-24')

480
Running query from 1 to 50
Running query from 51 to 100
Running query from 101 to 150
Running query from 151 to 200
Running query from 201 to 250
Running query from 251 to 300
Running query from 301 to 350
Running query from 351 to 400
Running query from 401 to 450
Running query from 451 to 500
All urls have been extracted successfully


### Looked at the first and last movie, and both matched query. Now going page by page to check the first entry to see what is going wrong.

In [4]:
a = [i for i in range(1,451,50)]

In [5]:
c = [urls[i] for i in a]

In [6]:
c

['https://www.imdb.com/title/tt2386490',
 'https://www.imdb.com/title/tt9271164',
 'https://www.imdb.com/title/tt9389622',
 'https://www.imdb.com/title/tt7662118',
 'https://www.imdb.com/title/tt8764144',
 'https://www.imdb.com/title/tt8442346',
 'https://www.imdb.com/title/tt9553898',
 'https://www.imdb.com/title/tt10468110',
 'https://www.imdb.com/title/tt10777022']

### Open each url to compare with first page of each test result 

In [7]:
import webbrowser

In [28]:
for i in c:
    webbrowser.open_new(i)

In [30]:
urls[49]

'https://www.imdb.com/title/tt5969180'

### Problem seems to be on the first page. Will check those one by one to see what could be going wrong. 

In [8]:
e = [i for i in range(0,50)]

In [9]:
f = [urls[i] for i in e]

In [10]:
len(f)

50

In [36]:
for i in f:
    webbrowser.open_new_tab(i)

In [11]:
f[3]

'https://www.imdb.com/title/tt3038708'

### Problem seems to be at index 4: the movie 'Iron Sky' opens two urls, one for a previous version and one for the new version. I.e. one is completely out of scope from the query. I need to find that specific tag and figure out what the difference with it is.


In [12]:
url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2019-01-01,2019-01-24'

In [13]:
resp = urllib.request.urlopen(url)
soup = BeautifulSoup(resp, 'html.parser')
links = [a.get('href') for a in soup.find_all('a', href=True)]
pot_list = []
for link in links:
    if 'title/tt' in link:
        pot_list.append(link)

In [14]:
pot_list

['/title/tt6823368/',
 '/title/tt6823368/',
 '/title/tt6823368/vote?v=X;k=',
 '/title/tt2386490/',
 '/title/tt2386490/',
 '/title/tt2386490/vote?v=X;k=',
 '/title/tt6476140/',
 '/title/tt6476140/',
 '/title/tt6476140/vote?v=X;k=',
 '/title/tt3038708/',
 '/title/tt3038708/',
 '/title/tt3038708/vote?v=X;k=',
 '/title/tt1034314',
 '/title/tt5886046/',
 '/title/tt5886046/',
 '/title/tt5886046/vote?v=X;k=',
 '/title/tt7985692/',
 '/title/tt7985692/',
 '/title/tt7985692/vote?v=X;k=',
 '/title/tt6811018/',
 '/title/tt6811018/',
 '/title/tt6811018/vote?v=X;k=',
 '/title/tt7458762/',
 '/title/tt7458762/',
 '/title/tt7458762/vote?v=X;k=',
 '/title/tt7634968/',
 '/title/tt7634968/',
 '/title/tt7634968/vote?v=X;k=',
 '/title/tt5598292/',
 '/title/tt5598292/',
 '/title/tt5598292/vote?v=X;k=',
 '/title/tt7616798/',
 '/title/tt7616798/',
 '/title/tt7616798/vote?v=X;k=',
 '/title/tt6563576/',
 '/title/tt6563576/',
 '/title/tt6563576/vote?v=X;k=',
 '/title/tt9288776/',
 '/title/tt9288776/',
 '/title/tt

### From the list above, it looks like movies that have a 'prequel' displays both entries, but only need the one. The entry that isn't actually part of the search only comes up once, so will set conditionals.

In [15]:
pot_list[2].split('/')

['', 'title', 'tt6823368', 'vote?v=X;k=']

### Run final test on two different searches to confirm that it is running correctly.

In [17]:
urls_1 = url_extractor('https://www.imdb.com/search/title/?title_type=feature&release_date=2011-09-12,2012-11-28&genres=action')

1043
Running query from 1 to 50
Running query from 51 to 100
Running query from 101 to 150
Running query from 151 to 200
Running query from 201 to 250
Running query from 251 to 300
Running query from 301 to 350
Running query from 351 to 400
Running query from 401 to 450
Running query from 451 to 500
Running query from 501 to 550
Running query from 551 to 600
Running query from 601 to 650
Running query from 651 to 700
Running query from 701 to 750
Running query from 751 to 800
Running query from 801 to 850
Running query from 851 to 900
Running query from 901 to 950
Running query from 951 to 1000
Running query from 1001 to 1050
All urls have been extracted successfully


In [18]:
urls_2 = url_extractor('https://www.imdb.com/search/title/?title_type=feature&release_date=2011-01-01,2012-11-28&genres=comedy,drama')

1222
Running query from 1 to 50
Running query from 51 to 100
Running query from 101 to 150
Running query from 151 to 200
Running query from 201 to 250
Running query from 251 to 300
Running query from 301 to 350
Running query from 351 to 400
Running query from 401 to 450
Running query from 451 to 500
Running query from 501 to 550
Running query from 551 to 600
Running query from 601 to 650
Running query from 651 to 700
Running query from 701 to 750
Running query from 751 to 800
Running query from 801 to 850
Running query from 851 to 900
Running query from 901 to 950
Running query from 951 to 1000
Running query from 1001 to 1050
Running query from 1051 to 1100
Running query from 1101 to 1150
Running query from 1151 to 1200
Running query from 1201 to 1250
All urls have been extracted successfully
